0. 所有的题目结果中，给出SQL语句和执行结果。

1. 在新数据库中新建一张 user 表,插入几条数据,属性包含:唯一标识(id),姓名(name)性别(sex).年龄(age).联系方式(phone)，数据如下：

    ('John Doe', 'Male', 25, '123-456-7890')

    ('Jane Smith', 'Female', 31, '987-654-3210')
    
    ('Bob Johnson', 'Male', 22, '555-123-4567')

In [1]:
import pymysql

# 连接到MySQL数据库 (使用root用户连接)
connection = pymysql.connect(
    host='localhost',      # 数据库主机
    user='root',           # 数据库用户名
    password='password'    # 数据库密码
)

cursor = connection.cursor()
# 创建数据库
cursor.execute("CREATE DATABASE IF NOT EXISTS test_db")
# 切换到创建的数据库
cursor.execute("USE test_db")
cursor.execute("drop table if exists user;")
# 创建user表
create_table_query = '''
CREATE TABLE IF NOT EXISTS user (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    sex VARCHAR(10),
    age INT,
    phone VARCHAR(20)
)
'''
cursor.execute(create_table_query)

# 插入数据
insert_data_query = '''
INSERT INTO user (name, sex, age, phone) 
VALUES 
('John Doe', 'Male', 25, '123-456-7890'),
('Jane Smith', 'Female', 31, '987-654-3210'),
('Bob Johnson', 'Male', 22, '555-123-4567')
'''
cursor.execute(insert_data_query)

# 提交事务
connection.commit()

# 查询数据并输出表格
cursor.execute("SELECT * FROM user")
results = cursor.fetchall()
for row in results:
    print(row)

# 关闭连接
cursor.close()
connection.close()


(1, 'John Doe', 'Male', 25, '123-456-7890')
(2, 'Jane Smith', 'Female', 31, '987-654-3210')
(3, 'Bob Johnson', 'Male', 22, '555-123-4567')


2. 写出 SQL语句,查询 user 表中所有年龄在 20-30 范围内的用户

In [ ]:
connection = pymysql.connect(
    host='localhost',      # 数据库主机
    user='root',           # 数据库用户名
    password='password',    # 数据库密码
    database='test_db'
)

# 创建新数据库
cursor = connection.cursor()

In [3]:
select_query_2 = "SELECT * FROM user WHERE age BETWEEN 20 AND 30;"
cursor.execute(select_query_2)
results_2 = cursor.fetchall()
for row in results_2:
    print(row)

(1, 'John Doe', 'Male', 25, '123-456-7890')
(3, 'Bob Johnson', 'Male', 22, '555-123-4567')


3. 写出SQL语句，向user表中添加自己的个人信息，并添加几条和你姓名同姓的虚拟信息。

In [5]:
sql = """
INSERT INTO user (id, name, sex, age, phone) VALUES
    (4, 'Xinran Yan', 'Female', 20, '555-000-1111'),
    (5, 'Moon Yan', 'Female', 30, '555-111-2222'),
    (6, 'Earth Yan', 'Female', 15, '555-222-3333');
""" 
cursor.execute("SELECT * FROM user")
results = cursor.fetchall()
for row in results:
    print(row)

(1, 'John Doe', 'Male', 25, '123-456-7890')
(2, 'Jane Smith', 'Female', 31, '987-654-3210')
(3, 'Bob Johnson', 'Male', 22, '555-123-4567')
(4, 'Xinran Yan', 'Female', 20, '555-000-1111')
(5, 'Moon Yan', 'Female', 30, '555-111-2222')
(6, 'Earth Yan', 'Female', 15, '555-222-3333')


4. 写出 SQL语句,查询 user 表中年龄在 20-30 范围内,名字包含“你的姓氏”的用户,并按照年龄从大到小排序输出

In [6]:
sql = "SELECT * FROM user WHERE age BETWEEN 20 AND 30 AND name LIKE '%Yan%' ORDER BY age DESC;"
cursor.execute(sql)
results = cursor.fetchall()
for row in results:
    print(row)

(5, 'Moon Yan', 'Female', 30, '555-111-2222')
(4, 'Xinran Yan', 'Female', 20, '555-000-1111')


5. 写出 SQL 语句,计算 user 表中所有用户的平均年龄

In [8]:
sql = "SELECT AVG(age) AS average_age FROM user;"
cursor.execute(sql)
result = cursor.fetchone()
print(f"Average Age: {result[0]}")

Average Age: 23.8333


6. 新建两张表team 表(id,teamName)和score 表(id,teamid,userid,score)。其中score 表中的 teamid 为指向 team表id 的外键，userid 为指向 user表id的外键

In [9]:
cursor.execute("""
CREATE TABLE team (
    id INT AUTO_INCREMENT PRIMARY KEY,
    teamName VARCHAR(255)
);
""")
cursor.execute("""
CREATE TABLE score (
    id INT AUTO_INCREMENT PRIMARY KEY,
    teamid INT,
    userid INT,
    score INT,
    FOREIGN KEY (teamid) REFERENCES team(id),
    FOREIGN KEY (userid) REFERENCES user(id)
);
""")
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
for table in tables:
    print(table)

('score',)
('team',)
('user',)


7. 在team表中和score表中插入合适的记录，写出 SQL语句,查询 teamName 为“ECNU”的队伍中，年龄小于 20 的用户们，结果不得为空。

In [11]:
cursor.execute("""
INSERT INTO team (teamName) VALUES ('ECNU'), ('PKU');
""")
cursor.execute("""
INSERT INTO score (teamid, userid, score) VALUES
    (1, 5, 95),
    (1, 6, 88),
    (2, 2, 76);
""")
select_query_7 = """
SELECT u.* 
FROM user u 
JOIN score s ON u.id = s.userid
JOIN team t ON s.teamid = t.id
WHERE t.teamName = 'ECNU' AND u.age < 20;
"""
cursor.execute(select_query_7)
results_7 = cursor.fetchall()
for row in results_7:
    print(row)


(6, 'Earth Yan', 'Female', 15, '555-222-3333')


8. 写出 SQL 语句,计算 teamName为“ECNU”的总分(假设 score 存在 null值,nul值默认为 0 加入计算)。

In [15]:
select_query_8 = """
SELECT SUM(COALESCE(s.score, 0)) AS total_score 
FROM score s 
JOIN team t ON s.teamid = t.id 
WHERE t.teamName = 'ECNU';
"""
cursor.execute(select_query_8)
result_8 = cursor.fetchone()

print(f"Total Score for ECNU: {result_8[0]}")

Total Score for ECNU: 183


9. 写出SQL语句，删除user表中个人信息的记录。

In [16]:
cursor.execute("DELETE FROM user WHERE id = 4;")
cursor.execute("select * from user;")
result = cursor.fetchall()
for row in result:
    print(row)

(1, 'John Doe', 'Male', 25, '123-456-7890')
(2, 'Jane Smith', 'Female', 31, '987-654-3210')
(3, 'Bob Johnson', 'Male', 22, '555-123-4567')
(5, 'Moon Yan', 'Female', 30, '555-111-2222')
(6, 'Earth Yan', 'Female', 15, '555-222-3333')


In [17]:
# 提交事务
connection.commit()

# 关闭连接
cursor.close()
connection.close()